<a href="https://colab.research.google.com/github/maxreynolds/Pattern-Cloud/blob/main/StreamlitImageUpload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup code
!cd /content/
!pip install -q streamlit
!pip install -q streamlit-image-select
!git clone https://github.com/huggingface/diffusers.git
!pip install -q ./diffusers
!pip install -q  -U -r /content/diffusers/examples/text_to_image/requirements.txt
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

In [ ]:
!mkdir /content/user_images
!ls

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

In [9]:
!streamlit run /content/app.py &>/content/logs.txt &

In [20]:
%%writefile app.py
from diffusers import StableDiffusionPipeline
from datasets import load_dataset

import streamlit as st
import torch

from PIL import Image
import csv
import os


def delete_files_in_directory(directory_path):
   try:
     files = os.listdir(directory_path)
     for file in files:
       file_path = os.path.join(directory_path, file)
       if os.path.isfile(file_path):
         os.remove(file_path)
   except OSError:
     print("Error occurred while deleting files.")

image_set = []
image_set = st.file_uploader("Upload a folder of your training dataset Images:",
    type=["png"], accept_multiple_files=True) # TODO: expand to type=["png","jpg","jpeg"]]... need to update metadata.csv writer code to check for filetype

if image_set is not None:
    #st.write('Scroll down for next steps')
    st.image(image_set, width=128)

if st.button('Upload Images to HuggingFace Hub', type="primary", use_container_width=True):
    placeholder = st.empty()
    placeholder.text("Formatting images...")
    for idx, image in enumerate(image_set):
      im = Image.open(image)
      im.save(f"/content/user_images/image{idx}.png")
    num_images = len(image_set)
    my_label = "<r4nd0m-l4b3l-1>"
    with open('/content/user_images/metadata.csv', 'w', newline='') as csvfile:
      spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
      spamwriter.writerow(['file_name,text'])
      for j in range(num_images):
        spamwriter.writerow([f'image{j}.png,'] + [my_label])
    placeholder.text('Loading dataset...')
    dataset = load_dataset("imagefolder", data_dir="/content/user_images", drop_labels=True)
    placeholder.text('Pushing to Hub...')
    dataset.push_to_hub("MaxReynolds/MyPatternDataset", private=False)
    placeholder.text('Upload complete!')
    #removing image files here
    delete_files_in_directory('/content/user_images')





Overwriting app.py
